# 线段树
- 着重解决区间问题，最经典的线段树问题：区间染色
- 解决更新、查询问题的时间复杂度都是O(logn)的，比用数组效率高很多(O(n))
- 线段树是一颗满树，所以必然是一颗完全二叉树，同时也是一颗平衡二叉树（最大深度和最小深度的高度差不超过1），注意完全二叉树这个性质很关键,这样就可以像maxheap那样来用数组组织树啦

# 用数组表示这颗树需要开多大空间？
- 考虑有h层（0——h-1）的二叉树
- 共有节点数量：$\frac{2^0(1-2^h)}{1-2}$ = $2^h-1$ = $\approx2^h$
- 最后一层节点的最大数量：$2^{h-1}$
- 得到除了最后一层前h-2层的节点数总量：$2^h-2^{h-1}=2^{h-1}$
- 所以最后一层的节点数量大致等于前面所有层节点之和（实则差1，当然可以忽略）
- 得到：如果区间有n个元素，若$n=2^k$，那么最底层必有n个元素，根据上面的推导，前面也有k个元素，所以数组要开辟$2n$个空间
- 但若$2^k<n<2^{k+1}$，那么此时必须要再开一层
- 由于线段树是一棵满树，这一层的数量等于前面层的节点只和，也就是$2n$
- 综上，数组需要开辟的空间为$2n+2n=4n$，也就是总共$4n$这么多个空间，才能保证将所有元素一定能够装进树中

### 个人认为线段树是一个非常有趣的数据结构，一定要好好消化、理解。
- 三个索引：


1. 树上节点的索引
2. 树上每个节点所代表的区间的索引
3. 对用户而言的data数组上的索引


- 理解好上面三个索引就能更好的理解线段树啦

In [1]:
# Time 2019-04-05 23:32
from collections import Iterable
class SegmentTree:
    def __init__(self, iterable, func):
        """
        线段树的构造函数
        Params:
            - iterable: 一个可叠戴对象，所以都可以转成python list
            - func: 初始化self.merger的函数
        """
        assert isinstance(iterable, Iterable)
        self.data = list(iterable)  # iterable是用户传进来的可迭代数据，我们把他转换成数组
        # 这是我们要根据用户传进来的数组组织成一个线段树的数组，注意是4n的空间
        self.tree = ['nan'] * len(self.data) * 4 
        self.merger = func # 一个函数，比如求和函数神马的
        self._buildSegmentTree(0, 0, len(self.data) - 1) # 从树的索引0开始对data的[0, len(data)-1]索引区间上的元素进行划分
        
    def getSize(self):
        """获取有效元素的数目"""
        return len(self.data)
    
    def get(self, index):
        """
        获取某一索引处的元素的值
        O(1)
        Returns:
            该索引上元素的值
        """
        return self.data[index] # python list自动做检查了
    
    def query(self, query_l, query_r):
        """
        查询data上某一区间被执行self.merger功能后的结果值（比如求和），注意区间是左闭右闭的
        O(logn)
        Params:
            - query_l: self.data数组上查询范围的左侧索引值
            - query_r: self.data数组上查询范围的右侧索引值
        Returns:
            merger该区间后的结果（比如求和）
        """
        # 用户传入的想要查询的区间，左闭右闭的
        # 安全检查 python list 自己会做的
        return self._query(0, 0, self.getSize() - 1, query_l, query_r)
    
    def set(self, index, e):
        """
        将self.data上某一索引位置的元素替换成另一个元素，注意要同时维护self.tree成员变量
        O(logn)
        Params:
            - index: 输入的索引
            - e: 将要替换的结果值
        """
        # 更新
        self.data[index] = e # 安全检查python list 做了
        # 维护self.tree
        self._set(0, 0, self.getSize() - 1, index, e)
    
    def print_(self):
        """对线段树中的元素进行打印，如果值非法的话这里显示为null"""
        print('[', end=' ')
        for elem in self.tree:
            if elem == 'nan':
                print('null', end=', ')
            else:
                print(elem, end=', ')
        print(']')
    
    # private
    # 对于线段树，不需要去找父节点
    def _leftChild(self, index):
        """
        根据当前索引去找其左孩子的索引
        Params:
            - index: 输入的索引
        Returns:
            左孩子的索引
        """
        return index * 2 + 1
    
    def _rightChild(self, index):
        """
        根据当前索引去找其右孩子的索引
        Params:
            - index: 输入的索引
        Returns:
            右孩子的索引
        """
        return index * 2 + 2
    
    def _buildSegmentTree(self, tree_index, data_l_index, data_r_index):
        """
        以线段树上的某一个索引作为根节点，来构建数组上某一个区间内的相应的线段树，注意区间是左闭右闭的
        Params:
            - tree_index: 线段树当前的根节点索引
            - data_l_index: 数组的左侧索引
            - data_r_index: 数组的右侧索引
        """
        # 注意是左闭右闭区间
        if data_l_index == data_r_index:
            self.tree[tree_index] = self.data[data_l_index]
            return
        mid_index = data_l_index + (data_r_index - data_l_index) // 2
        left_child_index = self._leftChild(tree_index)
        right_child_index = self._rightChild(tree_index)
        self._buildSegmentTree(left_child_index, data_l_index, mid_index)
        self._buildSegmentTree(right_child_index, mid_index + 1, data_r_index)
        
        # 回归过程中用过merger确定上层树节点所携带的值
        self.tree[tree_index] = self.merger(self.tree[left_child_index], self.tree[right_child_index])
        
    def _query(self, tree_index, tree_l_index, tree_r_index, data_l_index, data_r_index):
        """
        以线段树上的某一个索引作为根节点，来查询数组上某一区间的merger后的值，注意区间是左闭右闭的
        Params:
            - tree_index: 线段树当前的根节点索引
            - tree_l_index: 线段树当前的根节点所代表的索引范围的左边界
            - tree_r_index: 线段树当前的根节点所代表的索引范围的右边界
            - data_l_index: 待查询data数组上的区间的左边界
            - data_r_index: 待查询data数组上的区间的右边界
        Returns:
            data数组上[data_l_index, data_r_index]的元素merger后的值（比如求和）
        """
        # 在以tree_index所代表的区间[tree_l_index, tree_r_index]为根的线段树中，搜索区间[data_l_index, data_r_index]
        # merger后的值
        if tree_l_index == data_l_index and tree_r_index == data_r_index:
            return self.tree[tree_index]
        
        tree_mid_index = tree_l_index + (tree_r_index - tree_l_index) // 2
        left_child_index = self._leftChild(tree_index)
        right_child_index = self._rightChild(tree_index)
        if data_r_index <= tree_mid_index:
            return self._query(left_child_index, tree_l_index, tree_mid_index, data_l_index, data_r_index)
        elif data_l_index > tree_mid_index:
            return self._query(right_child_index, tree_mid_index + 1, tree_r_index, data_l_index, data_r_index)
        else:
            return self.merger(
                self._query(left_child_index, tree_l_index, tree_mid_index, data_l_index, tree_mid_index),
                self._query(right_child_index, tree_mid_index + 1, tree_r_index, tree_mid_index + 1, data_r_index)
            )
        
    def _set(self, tree_index, tree_l_index, tree_r_index, index, e):
        """
        将data数组上某一个索引位置的值替换成新值，同时对线段树进行相应的维护
        Params:
            - tree_index: 线段树当前的根节点索引
            - tree_l_index: 线段树当前的根节点所代表的索引范围的左边界
            - tree_r_index: 线段树当前的根节点所代表的索引范围的右边界
            - index: 发生替换操作的data数组上索引的位置
            - e: 待替换的新值
        """
        if tree_l_index == tree_r_index:
            self.tree[tree_index] = e
            return 
        tree_mid_index = tree_l_index + (tree_r_index - tree_l_index) // 2
        left_child_index = self._leftChild(tree_index)
        right_child_index = self._rightChild(tree_index)
        if index <= tree_mid_index:
            self._set(left_child_index, tree_l_index, tree_mid_index, index, e)
        else:
            self._set(right_child_index, tree_mid_index + 1, tree_r_index, index, e)
        
        # 回归的过程中需要通过merger函数对树的上层节点也进行相应的更新
        self.tree[tree_index] = self.merger(self.tree[left_child_index], self.tree[right_child_index])

In [2]:
# test segment_tree
nums = [-2, 0, 3, -5, 2, -1]
print('将nums中的元素添加进merger函数为两元素求和的线段树中-----', end=' ')
test = SegmentTree(nums, func=lambda x1, x2: x1 + x2) # 这里就是简单的区间求和函数
test.print_()
print('nums数组中[0, 2]左闭右闭区间上元素的和-----', test.query(0, 2))
print('nums数组中[2, 5]左闭右闭区间上元素的和-----', test.query(2, 5))
print('nums数组中[0, 5]左闭右闭区间上元素的和-----', test.query(0, 5))
print('将索引2的元素设为77-----', end=' ')
test.set(2, 77)
test.print_()
print('nums数组中[0, 2]左闭右闭区间上元素的和-----', test.query(0, 2))
print('nums数组中[2, 5]左闭右闭区间上元素的和-----', test.query(2, 5))
print('nums数组中[0, 5]左闭右闭区间上元素的和-----', test.query(0, 5))

将nums中的元素添加进merger函数为两元素求和的线段树中----- [ -3, 1, -4, -2, 3, -3, -1, -2, 0, null, null, -5, 2, null, null, null, null, null, null, null, null, null, null, null, ]
nums数组中[0, 2]左闭右闭区间上元素的和----- 1
nums数组中[2, 5]左闭右闭区间上元素的和----- -1
nums数组中[0, 5]左闭右闭区间上元素的和----- -3
将索引2的元素设为77----- [ 71, 75, -4, -2, 77, -3, -1, -2, 0, null, null, -5, 2, null, null, null, null, null, null, null, null, null, null, null, ]
nums数组中[0, 2]左闭右闭区间上元素的和----- 75
nums数组中[2, 5]左闭右闭区间上元素的和----- 73
nums数组中[0, 5]左闭右闭区间上元素的和----- 71
